In [1]:
import pandas as pd
from methods import data_handler
from tqdm import tqdm

from torchvision import datasets
import os
from itertools import chain


Bad key "text.kerning_factor" on line 4 in
/Users/julianvonklitzing/opt/anaconda3/envs/xai_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/julianvonklitzing/opt/anaconda3/envs/xai_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read questionnaire information from picking procedure

In [6]:
questionnaires = data_handler.get_questionaires("data2/questionaires_shuffled.pickle")
labels = data_handler.get_labels()

Using downloaded and verified file: ./data/imagenet_class_index.json


In [33]:
def get_label_from_img_idx(img_idx, testset_path, labels):    
    img_folder = datasets.ImageFolder(root=testset_path)
    img_path = img_folder.imgs[img_idx][0]
    img_name = img_path.split(os.sep)[-1]
    # extract correct class
    class_idx_true_str = img_path.split(os.sep)[-2]
    img_label_true = labels[class_idx_true_str][1]
    return img_label_true

In [34]:
# enrich questionnaire data with image label names
questionnaires_2 = []
for questionnaire in tqdm(questionnaires):
    questionnaire_2 = []
    for question in questionnaire:
        label = get_label_from_img_idx(question[0], "data2/imagenetv2-matched-frequency-format-val", labels)
        question_labled = (label, ) + question        
        questionnaire_2.append(question_labled)
    questionnaires_2.append(questionnaire_2) 

100%|███████████████████████████████████████████| 12/12 [00:36<00:00,  3.03s/it]


## Load question codes used in SoSci

In [36]:
codes_list = []
for i in range(1, 12+1):
    codes = pd.read_csv(f"questionaires_shuffle_order/questionaire_{i}.txt", sep=";", names=[0, 1])[1]
    codes = codes.str.extract("(\w\d{3})")
    codes = list(codes[0])
    codes_list.append(codes)

In [68]:
codes_list

[['B108',
  'B111',
  'B118',
  'B123',
  'B124',
  'B115',
  'B113',
  'B104',
  'B110',
  'B122',
  'B117',
  'B112',
  'B121',
  'B114',
  'B103',
  'B105',
  'B109',
  'B106',
  'B101',
  'B120',
  'B119',
  'B116',
  'B102',
  'B107'],
 ['B210',
  'B206',
  'B221',
  'B217',
  'B212',
  'B218',
  'B203',
  'B213',
  'B219',
  'B223',
  'B208',
  'B220',
  'B207',
  'B209',
  'B205',
  'B204',
  'B215',
  'B201',
  'B222',
  'B224',
  'B202',
  'B211',
  'B216',
  'B214'],
 ['B313',
  'B310',
  'B309',
  'B320',
  'B319',
  'B321',
  'B303',
  'B301',
  'B302',
  'B312',
  'B306',
  'B317',
  'B314',
  'B304',
  'B315',
  'B318',
  'B324',
  'B308',
  'B323',
  'B307',
  'B316',
  'B311',
  'B305',
  'B322'],
 ['B411',
  'B404',
  'B406',
  'B424',
  'B412',
  'B402',
  'B421',
  'B423',
  'B416',
  'B418',
  'B415',
  'B407',
  'B419',
  'B403',
  'B408',
  'B414',
  'B401',
  'B420',
  'B413',
  'B417',
  'B409',
  'B422',
  'B405',
  'B410'],
 ['B521',
  'B518',
  'B511',
  'B51

## Create questions meta data df

In [92]:
df_quest_meta = pd.DataFrame(list(chain(*questionnaires_2)))
df_quest_meta[5] = list(chain(*codes_list))
df_quest_meta.columns = ["label", "img_idx", "model", "method", "is_pred_correct", "question_code"]

In [93]:
df_quest_meta

,label,img_idx,model,method,is_pred_correct,question_code
0,custard_apple,9521,vgg,LRP,True,B108
1,Boston_bull,1073,vgg,SHAP,True,B111
2,toilet_tissue,9992,alex,IntegratedGradients,False,B118
3,gas_pump,5253,vgg,ConfidenceScores,False,B123
4,mailbag,5976,vgg,IntegratedGradients,False,B124
...,...,...,...,...,...,...
283,custard_apple,9521,vgg,gradCAM,True,C307
284,coral_reef,9710,alex,gradCAM,True,C302
285,black_grouse,7798,alex,LRP,False,C319
286,steel_arch_bridge,8034,alex,SHAP,True,C317


## Load and transform questionnaire data

In [94]:
df = pd.read_excel("data2/data_tu-helpfulness-of-xai_2022-06-21_17-15.xlsx")
df = df.drop(0)

In [95]:
def convert_q_data_from_wide_to_long(df):
    df_long = pd.melt(df, id_vars="CASE", value_vars=df.columns.values[6:294]).dropna()
    df_long.columns = ["case", "question_code", "response"]
    df_long = df_melt.sort_values("case", )
    # map response 1(Yes)/2(No) values to True/False
    df_long["response"] = df_long["response"].apply(lambda x: True if x==1 else False) 
    return df_long

In [96]:
df_long = convert_q_data_from_wide_to_long(df)

In [97]:
df_long

,case,question_code,response
19723,579,B720,True
17545,579,B702,False
19481,579,B718,False
18997,579,B714,True
19360,579,B717,True
...,...,...,...
13067,1566,B512,False
12583,1566,B508,True
13672,1566,B517,True
12825,1566,B510,True


## Calculate metrics

In [104]:
df_merged = df_long.merge(right=df_quest_meta)
df_merged = df_merged.sort_values("case")

In [105]:
df_merged

,case,question_code,response,label,img_idx,model,method,is_pred_correct
0,579,B720,True,cucumber,9389,vgg,gradCAM,True
45,579,B701,False,water_tower,8917,vgg,SHAP,True
54,579,B719,True,mortar,6303,alex,LRP,False
117,579,B721,True,beagle,712,vgg,LRP,False
63,579,B705,True,jaguar,2136,vgg,LIME,True
...,...,...,...,...,...,...,...,...
2455,1566,B504,False,shopping_cart,7692,alex,LIME,True
2463,1566,B503,False,custard_apple,9521,vgg,IntegratedGradients,True
2471,1566,B502,False,matchstick,6063,vgg,SHAP,False
2383,1566,B519,True,Weimaraner,880,vgg,gradCAM,False


### Overall ability to guess model is correct if actually correct

In [110]:
num_guessed_correct_if_correct = df_merged[(df_merged["is_pred_correct"] == True) & (df_merged["response"] == True)].shape[0]
num_correct_overall = df_merged[df_merged["is_pred_correct"] == True].shape[0]

In [111]:
num_guessed_correct_if_correct / num_correct_overall

0.6308539944903582

### Overall ability to guess model is wrong if actually wrong

In [112]:
num_guessed_wrong_if_wrong = df_merged[(df_merged["is_pred_correct"] == False) & (df_merged["response"] == False)].shape[0]
num_wrong_overall = df_merged[df_merged["is_pred_correct"] == False].shape[0]

In [113]:
num_guessed_wrong_if_wrong / num_wrong_overall

0.5475206611570248

### Overall ability to guess model is correct if actually correct